In [ ]:
# IMPORT THE LIBRARIES
from ctypes import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tkinter import *
from PIL import ImageTk, Image
import cv2
from tifffile import imsave
import os
import time
import ctypes
import ctypes.wintypes

# INITIALIZE THORLABS CAMERAS

#LOAD the driver
uc480_file = 'C:\\Program Files\\Thorlabs\\Scientific Imaging\\DCx Camera Support\\USB Driver Package\\uc480_64.dll'
uc480 = windll.LoadLibrary(uc480_file)
is_InitCamera = uc480.is_InitCamera

#OPEN the CMOS
is_InitCamera.argtypes = [POINTER(c_int)]
handle1 = c_int(0)
i1 = is_InitCamera(byref(handle1))   
if i1 == 0: 
    print('CMOS1 open')
else :
    print('échec de l''ouverture CMOS1')

    
#set the exposure and pixel clock
exposure=20
exposure_c = c_double(exposure)
is_Exposure = uc480.is_Exposure
is_Exposure.argtypes = [c_int, c_uint, POINTER(c_double), c_uint]
is_Exposure(handle1, 12 , exposure_c, 8) #12 is for setting exposure
exposure = exposure_c.value

pixelclock = c_uint(43) 
is_PixelClock = uc480.is_PixelClock
is_PixelClock.argtypes = [c_int, c_uint, POINTER(c_uint), c_uint]
is_PixelClock(handle1, 6 , byref(pixelclock), sizeof(pixelclock)) #6 for setting pixel clock
uc480.is_SetColorMode(handle1, 26)
uc480.is_SetExternalTrigger(handle1,1)

 
#allocate memory
xdim=5808
ydim=1216
imagesize = xdim*ydim
memid = c_int(0)
c_buf = (c_ubyte * imagesize)(0)
uc480.is_SetAllocatedImageMem(handle1, xdim, ydim, 8, c_buf, byref(memid))
uc480.is_SetImageMem(handle1, c_buf, memid)
meminfo = [c_buf, memid]

 

def openCMOS():
    is_InitCamera = uc480.is_InitCamera
    is_InitCamera.argtypes = [POINTER(c_int)]
    handle1 = c_int(0)
    i = is_InitCamera(byref(handle1))   
    if i == 0: 
        print('CMOS 1 open')
    else :
        print('échec de l''ouverture CMOS 1')

        
def closeCMOS():
    handle=1
    i = uc480.is_ExitCamera(handle1) 
    if i == 0: 
        print('CMOS1 fermée')
    else :
        print('échec de la fermeture CMOS 1')


def SetFPS1():
    uc480.is_StopLiveVideo(handle1,1)
    FPS1=float(entry_FPS1.get())
    FPS1c = c_double(FPS1)
    newFPS=ctypes.c_double(0)
    uc480.is_SetFrameRate (handle1, FPS1c, ctypes.byref(newFPS))
    uc480.is_CaptureVideo(handle1,1)
    return newFPS

  

def AllocMem():
    xdim=5808
    ydim=1216
    imagesize = xdim*ydim
    memid = c_int(0)
    c_buf = (c_ubyte * imagesize)(0)
    uc480.is_SetAllocatedImageMem(handle1, xdim, ydim, 8, c_buf, byref(memid))
    uc480.is_SetImageMem(handle1, c_buf, memid)
    meminfo = [c_buf, memid]

    

def Snap1():
    xdim=5808
    ydim=1216
    imagesize = xdim*ydim
    memid = c_int(0)
    c_buf = (c_ubyte * imagesize)(0)
    uc480.is_SetAllocatedImageMem(handle1, xdim, ydim, 8, c_buf, byref(memid))
    uc480.is_SetImageMem(handle1, c_buf, memid)
    meminfo = [c_buf, memid]
    uc480.is_CaptureVideo(handle1,1)
    im1 = np.frombuffer(meminfo[0], c_ubyte).reshape(ydim, xdim)   
    return im1

def StopLive():
    uc480.is_StopLiveVideo(handle1,1)

     

def StartLive():
    uc480.is_CaptureVideo(handle1,1)

def SetExposure1a(a):
    uc480.is_StopLiveVideo(handle1,1)
    exposure_c1 = c_double(a)
    is_Exposure = uc480.is_Exposure
    is_Exposure.argtypes = [c_int, c_uint, POINTER(c_double), c_uint]
    is_Exposure(handle1, 12 , exposure_c1, 8) #12 is for setting exposure
    exposure1 = exposure_c1.value 
    uc480.is_CaptureVideo(handle1,1)

    

def SetExposure1():
    uc480.is_StopLiveVideo(handle1,1)
    exposure1=float(entry_exposure1.get())
    exposure_c1 = c_double(exposure1)
    is_Exposure = uc480.is_Exposure
    is_Exposure.argtypes = [c_int, c_uint, POINTER(c_double), c_uint]
    is_Exposure(handle1, 12 , exposure_c1, 8) #12 is for setting exposure
    exposure1 = exposure_c1.value 
    uc480.is_CaptureVideo(handle1,1)

    

def Save_snapshot():
    img1 = Image.fromarray(cv2.resize(II1, dsize=(1936,1216), interpolation=cv2.INTER_CUBIC))
    img1=np.asanyarray(img1)
    file_name = entry_filename.get()
    file_path = entry_directory.get()
    suffix = '.tif'
    total=os.path.join(file_path, file_name +'1' + suffix)
    imsave(total,img1) 

def video_capture():
    N=int(entry_nbframes.get())
    p=float(entry_timelapse.get())
    Vid=np.zeros((N,xdim,ydim))
    for i in range(N):
        img3=np.resize(II,(xdim,ydim))
        time.sleep(0.03+p)
        Vid[i,:,:]=img3
    file_name = entry_filename.get()
    file_path = entry_directory.get()
    suffix = '.tif'
    total=os.path.join(file_path, file_name + suffix)
    Vid = Vid.astype('int8')
    imsave(total,Vid)
    return Vid


closeCMOS()
openCMOS()
Snap1()
StopLive()

root = Tk()
root.title('GUI acquisition Thorlabs CMOS')
root.geometry("1500x800")
SetExposure1a(5)
II1=Snap1()
 

def video_stream1():
    img1 = Image.fromarray(cv2.resize(II1, dsize=(600,600), interpolation=cv2.INTER_CUBIC))
    imgtk1 = ImageTk.PhotoImage(image=img1)
    lmain1.imgtk = imgtk1    
    lmain1.configure(image=imgtk1)  
    lmain1.after(1, video_stream1) 

    

#TOP FRAME
frame0=Frame(root, width=1500, height=100, background="White")
Title=Label(frame0,text='DUAL VIEW MICROSCOPE - Acquisition software', background="white")
Title.config(font=('Arial', 18))
Title.grid(column=5,row=0,columnspan=3)



#LEFT FRAME - CMOS 1
frame1=Frame(root, width=500, height=700, background="mint cream")
lmain1 = Label(frame1, height=500,width=500)
lmain1.grid(row=1,column=0,rowspan=4,columnspan=4)

button_exposure1 = Button(frame1, text='Update XP time',bg='light blue', height=1, width=15, command = SetExposure1)
button_exposure1.grid(row=5, column=1)
entry_exposure1 = Entry(frame1)
entry_exposure1.insert(10,"1")
entry_exposure1.grid(row=5, column=3) 

button_FPS1 = Button(frame1, text='Update FPS',bg='light blue', height=1, width=15, command = SetFPS1)
button_FPS1.grid(row=6, column=1)

entry_FPS1 = Entry(frame1)
entry_FPS1.insert(10,"1")
entry_FPS1.grid(row=6, column=3) 
frame1.grid(row=1, column=0,rowspan=6, columnspan=4)


#RIGHT FRAME - ACQUISITION AND MOTION CONTROL
frame3=Frame(root, width=500, height=700, background="tomato")
button_start = Button(frame3, text='Start preview',bg='Seagreen1', height=2, width=10, command = StartLive)
button_start.grid(row=0, column=0,columnspan=1)
button_stop = Button(frame3, text='Stop preview',bg='firebrick1', height=2, width=10, command = StopLive)
button_stop.grid(row=0, column=1,columnspan=1)
button_snap = Button(frame3, text='Save snapshot',bg='Seagreen2', height=2, width=10, command = Save_snapshot)
button_snap.grid(row=0, column=2,columnspan=1)
text_directory = Label(frame3, text='Directory path', height=2, width=10,background="mint cream")
text_directory.grid(row=1, column=0)
entry_directory = Entry(frame3)
entry_directory.insert(10,"C:/Users/Admin")
entry_directory.grid(row=1, column=2)
text_filename = Label(frame3, text='File name', height=2, width=10,background="mint cream")
text_filename.grid(row=2, column=0)
entry_filename = Entry(frame3)
entry_filename.insert(10,"default")
entry_filename.grid(row=2, column=2)
frame3.grid(row=1, column=8,rowspan=6, columnspan=4)


#LAUNCH LOOP

video_stream1()
root.mainloop()